In [41]:
import csv
import numpy as np
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

class NHL_Period_Data:
    def __init__(self):
        self.data_x, self.data_y = self.__parseData__()
        
    def __parseData__(self):
        data = np.genfromtxt("data/periodStats.txt", delimiter=",", dtype=int, skip_header=1)
        return data[...,1:-2],data[...,-1]
    
    def transformData(self, encoder="StandardScaler"):
        col_tr = ColumnTransformer([(encoder, getattr(preprocessing,encoder)(), list(range(len(self.data_x[0]))))])
        col_tr.fit(self.data_x, self.data_y)
        self.data_x = col_tr.transform(self.data_x)
        
    def trainTestSplit(self, split=0.2):
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(self.data_x, self.data_y, test_size=split)
        
            

In [44]:
data = NHL_Period_Data()
print(data.data_x[0])
data.transformData("MinMaxScaler")
print(data.data_x[0])
data.trainTestSplit()

[ 0  7  6  9  5  4  6 15 11  3  4  2  0  6 10  4]
[0.46666667 0.31818182 0.33333333 0.40909091 0.2173913  0.26666667
 0.35294118 0.48387097 0.34375    0.2        0.21052632 0.08333333
 0.         0.2        0.35714286 0.30769231]
